# Recommender System with Deep Learning

## Setup

### Packages & Parameters

In [3]:
import os
import sys
import gc
import psutil
from random import randint
import time

import urllib.request
import multiprocessing as mp
import concurrent.futures

import gzip
import unicodedata
import math
import re

import pandas as pd
import numpy as np

from flair.models import TextClassifier
from flair.data import Sentence

### Globals

In [4]:
MAIN_DIR = os.path.join("D:" + os.sep, "Code", "PYTHON", "Amazon_Recommender_System")

CODE_DIR = os.path.join(MAIN_DIR, "Code")

ANALYSIS_DIR = os.path.join(MAIN_DIR, "Analysis")
DATA_DIR = os.path.join(MAIN_DIR, "Data")

RAW_DATA_DIR = os.path.join(DATA_DIR, "Raw")
CLEAN_DATA_DIR = os.path.join(DATA_DIR, "Clean")

BOOKS_RAW_DIR = os.path.join(RAW_DATA_DIR, "Books")
BOOKS_CLEAN_DIR = os.path.join(CLEAN_DATA_DIR, "Books")
BOOKS_SENTIMENT_DIR = os.path.join(BOOKS_CLEAN_DIR, "Sentiment")

### Directory Setup

In [5]:
def create_directory(dir_list: list) -> None:
    for directory in dir_list:
        if not os.path.exists(directory):
            os.makedirs(directory)

def is_directory_empty(dir_path: str) -> bool:
    return len(os.listdir(dir_path)) == 0

In [6]:
create_directory([CODE_DIR, 
                  ANALYSIS_DIR,
                  DATA_DIR, 
                  RAW_DATA_DIR, 
                  CLEAN_DATA_DIR,
                  BOOKS_RAW_DIR,
                  BOOKS_CLEAN_DIR,
                  BOOKS_SENTIMENT_DIR,
                  ])

## Data Processing

### Loading Data In

In [7]:
# More memory efficient version. Takes longer but is far more consistent than the previous version. 
def process_data(url: str, chunk_size: int, num_workers: int, output_dir: str) -> None:
    with urllib.request.urlopen(url) as response:
        with gzip.open(response, "rt") as gz_file:
            chunk = []
            chunk_count = 0

            with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
                for line in gz_file:
                    chunk.append(line)

                    if len(chunk) == chunk_size:
                        executor.submit(process_chunk, list(chunk), os.path.join(output_dir, f"chunk_{chunk_count}.json"))
                        chunk = []
                        chunk_count += 1

                # Process the remaining lines in the last chunk
                if chunk:
                    executor.submit(process_chunk, list(chunk), os.path.join(output_dir, f"chunk_{chunk_count}.json"))

def process_chunk(chunk: list, filename: str) -> None:
    with open(filename, 'w') as file:
        file.writelines(chunk)
    print(f"Processed chunk: {filename}")

In [8]:
'''
----- PROCESS_DATA -----
GAME URL "https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Video_Games_5.json.gz"
BOOK URL "https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Books_5.json.gz"

EXPERIMENTS FOR 27 MILLION
----- THREAD CSV_FILE -----
1. 100000 chunk -> 22min 26sec

----- THREAD JSON_FILE RUNTIMES-----
1. 100000 chunk -> 20min 55sec
2. 100000 chunk -> 49min 1sec -> No idea why this happened on a fresh start
3. 500000 chunk -> 18min 45sec
4. 1000000 chunk -> 17min 5sec

NEW STABLE IMPLEMENTATION
1. 1000000 chunk -> 22min 22sec
'''
url = "https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Books_5.json.gz"

raw_data_directory = BOOKS_RAW_DIR
clean_data_directory = BOOKS_CLEAN_DIR
sentiment_data_directory = BOOKS_SENTIMENT_DIR

chunk_size = 1000000

NUM_CORES = math.ceil(mp.cpu_count()/2)

CPU_ARGS = {
    "cpu_target" : 0.80, # Set some initial CPU load values as a CPU usage goal
    "thread_group_size" : NUM_CORES, # When CPU load is significantly low, start this number of threads
}

In [ ]:
process_data(url, chunk_size, NUM_CORES, raw_data_directory)

### Cleaning Data

In [9]:
def extract_value(dictionary: dict):
    return dictionary.get("Format:") if isinstance(dictionary, dict) else None

def filter_comment_length(reviews: pd.DataFrame, minimum: int) -> pd.DataFrame:
    reviews = reviews.copy()
    reviews["review_len"] = reviews["review_text"].str.split().str.len()
    reviews = reviews.loc[(reviews["review_len"] > minimum)]
    return reviews

def remove_irrelevant_info(reviews: pd.Series) -> pd.Series:
    return reviews.apply(lambda x: re.sub(r"http\S+|www.\S+|#\S+|<.*?>|\(|\)|\d+", "", x)) # Cleans up URL, hashtags, parenthesis, and numbers.

def reduce_characters(reviews: pd.Series) -> pd.Series:
    return reviews.apply(lambda x: re.sub(r"[^\w\s]|(.)\1+", "", x)) # Remove excessive punctuation and repeated characters

def normalize_encoding(reviews: pd.Series) -> pd.Series:
    return reviews.apply(lambda x: unicodedata.normalize("NFKD", x).encode("ASCII", "ignore").decode("utf-8")) # Encodes and Decodes the data so that we have consistency in text
        
def clean_chunk(df: pd.DataFrame) -> pd.DataFrame:

    df = df.rename(columns={
        "overall": "rating",
        "reviewTime": "review_date",
        "reviewerID": "reviewer_id",
        "asin": "product_id",
        "reviewText": "review_text",
    })

    df["vote"] = df["vote"].fillna(0)
    df["vote"] = pd.to_numeric(df["vote"].astype("str").str.replace(",","")).astype("int32")
    df = df[df["vote"] >= 5]
    df = filter_comment_length(df, 20)
    df.drop(["unixReviewTime", "image", "summary", "reviewerName"], axis=1, inplace=True)
    
    df["review_date"] = pd.to_datetime(df["review_date"], format="%m %d, %Y")
    df["style"] = df["style"].apply(extract_value)

    df["review_text"] = df["review_text"].str.lower()
    df["review_text"] = remove_irrelevant_info(df["review_text"])
    df["review_text"] = reduce_characters(df["review_text"])
    df["review_text"] = normalize_encoding(df["review_text"])
    return df

In [10]:
raw_file_names = [f"chunk_{file_num}.json" for file_num in range(len(os.listdir(raw_data_directory)))]

for raw_file_name in raw_file_names:
    raw_df_list = []
    raw_df = pd.read_json(os.path.join(raw_data_directory, raw_file_name), lines=True)
   
    raw_df = clean_chunk(raw_df)
    print(f"{raw_file_name} DIMENSIONS: {raw_df.shape}")
    clean_dir = os.path.join(clean_data_directory, raw_file_name)
    raw_df.to_json(clean_dir, orient="records")

    del raw_df
    gc.collect()

KeyboardInterrupt: 

## Sentiment Analysis

In [11]:
def batch_text(column: pd.Series, batch_size: int) -> list:
    batches = np.array_split(np.array(column.values), int(np.ceil(len(column)/ batch_size)))
    batches = [batch.tolist() for batch in batches]
    return batches

def load_sentiment_model(model: str):
    return TextClassifier.load(model)

def is_float(value):
      try:
        float(value)
        return True
      except ValueError:
        return False
      
def analyze_sentiment(model, batch: np.ndarray):
    score = []
    for sentence in batch:
        sentence = Sentence(sentence)
        model.predict(sentence, verbose=True)
        process = re.sub(r"\(|\)","",str(sentence.labels[0]))
        number = [float(s) for s in  process.split() if is_float(s) is True]
        if "POSITIVE" in process:
            score.append(number[0])
        elif "NEGATIVE" in process:
            score.append(-number[0])
    return score
   
def process_batches(models: list, batches: list):
    models_loaded = [load_sentiment_model(model) for model in models]
    
    sentiment_results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers= 4) as executor:
        sentiment_tasks = [executor.submit(analyze_sentiment, model, batch) for model, batch in zip(models_loaded, batches)]
        concurrent.futures.wait(sentiment_tasks)
        [sentiment_results.extend(task.result()) for task in sentiment_tasks]
        return sentiment_results

In [13]:
clean_file_names = [f"chunk_{file_num}.json" for file_num in range(len(os.listdir(raw_data_directory)))]

for clean_file_name in clean_file_names[0:1]:
    clean_df = pd.read_json(os.path.join(clean_data_directory, clean_file_name), orient="columns")[0:24]
    batch_size = math.ceil(len(clean_df)/2) 
    
    batches = batch_text(clean_df["review_text"], batch_size)
    # models_loaded = [load_sentiment_model(model) for model in ["en-sentiment"] * len(batches)]
    # clean_df["sentiment_score"] = process_batches(["en-sentiment"] * len(batches), batches)
    model = TextClassifier.load("en-sentiment")
    my_queue = 
    for batch in batches:
        batch
    start_thread_process(model, batches)
    # sentiment_dir = os.path.join(sentiment_data_directory, clean_file_name)
    # clean_df.to_json(sentiment_dir, orient="records")

['great day for up is a unique dr seu bk and you can te this just by lking at the coverthat is because while the bk is wrien by dr seu it features the joy drawings of the english artist quentin blakeuntil this point every time i have read a bk wrien by dr seu it was also iustrated by dr seu and when somebody else did the drawings dr seu used the name theo lesieg which is geisel backwardsso the fact that this is a real dr seu bk drawn by somebody else is prey special\nthis bright and early bk provides rhymed text and iustrations introducing the many meanings of the word up as seu and blake show begiing readers that this is a great day for upyou get the point half way through the bk but lile kids should be able to hand on longer especiay when they are reading the bk for themselvesbesides by the end of great day for up we get to the point where everyone on earth is up with one very important and rather ironic exception\nas with a of the bright and early bks for begiing begiers what you ha

In [ ]:
def analyze_sentiment(model, batch: np.ndarray):
    score = []
    for sentence in batch:
        sentence = Sentence(sentence)
        model.predict(sentence, verbose=True)
        process = re.sub(r"\(|\)","",str(sentence.labels[0]))
        number = [float(s) for s in  process.split() if is_float(s) is True]
        if "POSITIVE" in process:
            score.append(number[0])
        elif "NEGATIVE" in process:
            score.append(-number[0])
    return score

# Spool down the thread balance when load is too high
def spool_down_load_balance(args_array):
    one_minute_load_average = psutil.getloadavg()[0] / NUM_CORES # Calculate the short term load average of past minute
    # If load balance above the max return True to kill the process
    if one_minute_load_average > args_array['cpu_target']:
        print("-Unacceptable load balance detected. Killing process " + str(os.getpid()) + "...")
        return True

# Load balancer function
def load_balance(item_queue, args_array):
    print("[Starting load balancer...]")
    # While there are still items in the queue
    while not item_queue.empty():
        print("[Calculating load balance...]")
        one_minute_load_average = psutil.getloadavg()[0] / NUM_CORES # Check the 1 minute average CPU load balance and returns 1,5,15 minute load averages
        # If the load average is much less than the target, start a group of new threads
        if one_minute_load_average < args_array['cpu_target'] / 2:
            # Print message and log that load balancer is starting another thread group
            print("Starting another thread group due to low CPU load balance of: " + str(one_minute_load_average * 100) + "%")
            time.sleep(5)
        
            with concurrent.futures.ThreadPoolExecutor(max_workers=args_array['thread_group_size']) as executor:
                for _ in range(args_array['thread_group_size']):
                    executor.submit(analyze_sentiment, model, batch) 
            # Allow the added threads to have an impact on the CPU balance
            # before checking the one minute average again
            time.sleep(20)

        # If the load average is less than the target, start a single thread
        elif one_minute_load_average < args_array['cpu_target']:
            # Print message and log that load balancer is starting another thread
            print("Starting another single thread due to low CPU load balance of: " + str(one_minute_load_average * 100) + "%")
            # Start another thread
            with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
                executor.submit(main_process, item_queue.get())
            # Allow the added thread to have an impact on the CPU balance
            # before checking the one minute average again
            time.sleep(20)

        else:
            # Print CPU load balance
            print("Reporting stable CPU load balance: " + str(one_minute_load_average * 100) + "%")
            # Sleep for another minute
            time.sleep(20)